### Google Analytics 4 and BigQuery with Pandas
#### [GA4 Public Dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ga4_obfuscated_sample_ecommerce&t=events_20210131&page=table)


In [2]:
# Uncomment the below lines and run them if you haven't installed the module before
#%pip install google
#%pip install db-dtypes
#/\ NOT WORKING
#!pip install google-cloud-bigquery google-auth pandas


Note: you may need to restart the kernel to use updated packages.



In [12]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

# Use a service account for authentication onto GCP
credentials = service_account.Credentials.from_service_account_file('./secrets/ecommercedata-413417-6497dae93c8d.json')

# The project id to which the service account has access
project_id = 'ecommercedata-413417'

# Establish a connection to the client
client = bigquery.Client(credentials= credentials, project=project_id)

# Perform a query
query = client.query(
"""
SELECT
  event_date,
  event_timestamp,
  event_name,
  event_params.value.string_value AS page_url,
  device.*
FROM
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_20201101`,
  UNNEST(event_params) AS event_params --UNNEST allows us to extract the nested records
WHERE
  event_params.key = 'page_location'
""")

#Convert output into a pandas dataframe
df = query.to_dataframe()
df


,event_date,event_timestamp,event_name,page_url,category,mobile_brand_name,mobile_model_name,mobile_marketing_name,mobile_os_hardware_model,operating_system,operating_system_version,vendor_id,advertising_id,language,is_limited_ad_tracking,time_zone_offset_seconds,web_info
0,20201101,1604192073036492,page_view,https://shop.googlemerchandisestore.com/Google...,desktop,Google,Chrome,<Other>,<NA>,Web,10,<NA>,<NA>,zh,No,<NA>,"{'browser': 'Chrome', 'browser_version': '86.0'}"
1,20201101,1604192073036492,first_visit,https://shop.googlemerchandisestore.com/Google...,desktop,Google,Chrome,<Other>,<NA>,Web,10,<NA>,<NA>,zh,No,<NA>,"{'browser': 'Chrome', 'browser_version': '86.0'}"
2,20201101,1604192082302672,user_engagement,https://shop.googlemerchandisestore.com/Google...,desktop,Google,Chrome,<Other>,<NA>,Web,10,<NA>,<NA>,zh,No,<NA>,"{'browser': 'Chrome', 'browser_version': '86.0'}"
3,20201101,1604192073036492,session_start,https://shop.googlemerchandisestore.com/Google...,desktop,Google,Chrome,<Other>,<NA>,Web,10,<NA>,<NA>,zh,No,<NA>,"{'browser': 'Chrome', 'browser_version': '86.0'}"
4,20201101,1604202750089608,user_engagement,https://shop.googlemerchandisestore.com/,mobile,Samsung,<Other>,<Other>,<NA>,Android,Android 9,<NA>,<NA>,en-us,No,<NA>,"{'browser': 'Chrome', 'browser_version': '86.0'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31267,20201101,1604191178435076,view_item,https://shop.googlemerchandisestore.com/Google...,desktop,Apple,Safari,<Other>,<NA>,Web,Intel 10.15,<NA>,<NA>,en-gb,No,<NA>,"{'browser': 'Chrome', 'browser_version': '87.0'}"
31268,20201101,1604236810906150,view_item,https://shop.googlemerchandisestore.com/Google...,desktop,Apple,Safari,<Other>,<NA>,Macintosh,<Other>,<NA>,<NA>,ko,No,<NA>,"{'browser': 'Chrome', 'browser_version': '87.0'}"
31269,20201101,1604236804332730,view_item,https://shop.googlemerchandisestore.com/Google...,desktop,Apple,Safari,<Other>,<NA>,Macintosh,<Other>,<NA>,<NA>,ko,No,<NA>,"{'browser': 'Chrome', 'browser_version': '87.0'}"
31270,20201101,1604220772353910,view_item,https://shop.googlemerchandisestore.com/Google...,desktop,Google,Chrome,<Other>,<NA>,Windows,Windows 10,<NA>,<NA>,en-us,No,<NA>,"{'browser': 'Chrome', 'browser_version': '87.0'}"


In [13]:
name_values = list(set(df['event_name']))#.tolist()
print(name_values)

['add_shipping_info', 'page_view', 'add_to_cart', 'select_promotion', 'user_engagement', 'scroll', 'view_item_list', 'begin_checkout', 'add_payment_info', 'view_promotion', 'click', 'purchase', 'session_start', 'view_item', 'view_search_results', 'first_visit']


In [15]:
name_counts = df['event_name'].value_counts()
print(name_counts)

event_name
page_view              9767
user_engagement        8667
view_item              3380
scroll                 3040
session_start          2594
first_visit            1946
view_promotion         1263
view_search_results     194
begin_checkout          127
add_shipping_info       127
add_payment_info         73
select_promotion         57
purchase                 14
click                    11
view_item_list            8
add_to_cart               4
Name: count, dtype: int64
